# User Confirmation In Tools

In [1]:
from agentia import Agent, UserConsentEvent, AssistantMessage
from typing import Annotated
import dotenv

dotenv.load_dotenv()

True

### User Confirmation Before Running Sensitive/Important Actions

In [ ]:
def calc(expression: Annotated[str, "The expression to calculate"]):
    """Calculate the result of a mathematical expression"""

    if not (yield UserConsentEvent(message="Are you sure you want to calculate this?")):
        return "User did not consent to calculation."

    return eval(expression)


agent = Agent(tools=[calc])

run = agent.run("What is the result of 2 + 2?", events=True)

async for event in run:
    if isinstance(event, UserConsentEvent):
        # You need to handle the user consent event and respond accordingly
        print(f"Received consent event: {event.message}")
        while True:
            response = input(event.message + (" (y/n): ")).lower().strip()
            if response in ("y", "yes", "n", "no"):
                break
        print(f"User responded with: {response}")
        event.response = response in ("y", "yes")
    elif isinstance(event, AssistantMessage):
        print(event)

Received consent event: Are you sure you want to calculate this?
User responded with: y
AssistantMessage(content='The result of \\(2 + 2\\) is \\(4\\).', reasoning=None, tool_calls=[], role='assistant')


### For Async Tools

For async tools, it is not possible to return the result after making the tool an async generator.

Raise a `ToolResult` instance instead.

In [3]:
from agentia import ToolResult


async def calc(expression: Annotated[str, "The expression to calculate"]):
    """Calculate the result of a mathematical expression"""

    if not (yield UserConsentEvent("Are you sure you want to calculate this?")):
        raise ToolResult("User did not consent to calculation.")

    raise ToolResult(eval(expression))